In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chandra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# population is bigger and then take sample 
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
if not os.path.exists('sample_data.csv'):
    df.to_csv('sample_data.csv', index=False)
    df=pd.read_csv('sample_data.csv')
else:
    df=pd.read_csv('sample_data.csv')
df.head()

,review,sentiment
0,"Even if it were remotely funny, this mouldy wa...",negative
1,Writer/Director Peter Greenaway cements his ti...,negative
2,A ridiculous comedy given an arms-flailing dir...,negative
3,Set in and near a poor working class town in t...,positive
4,I wasn't really hoping for much when I went to...,negative


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
0,even remotely funny mouldy waxwork film would ...,negative
1,writer director peter greenaway cement title h...,negative
2,ridiculous comedy given arm flailing direction...,negative
3,set near poor working class town mountain rura...,positive
4,really hoping much went see this mstking heck ...,negative


In [5]:
# Balanced Data Set 
df['sentiment'].value_counts()

sentiment
negative    259
positive    241
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
# convert positive an negative---->>> 0 and 1
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,even remotely funny mouldy waxwork film would ...,0
1,writer director peter greenaway cement title h...,0
2,ridiculous comedy given arm flailing direction...,0
3,set near poor working class town mountain rura...,1
4,really hoping much went see this mstking heck ...,0


In [8]:
#check null values
df.isnull().sum()


review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
# train and test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [15]:

import dagshub
mlflow.set_tracking_uri('https://dagshub.com/chandrasekharcse522/Movie-Review-NLP-Pipeline.mlflow')
dagshub.init(repo_owner='chandrasekharcse522', repo_name='Movie-Review-NLP-Pipeline', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Movie-Review Logistic Regression Baseline")

# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "chandrasekharcse522/Movie-Review-NLP-Pipeline"

2025-09-24 13:26:06,600 - INFO - Initialized MLflow to track repo "chandrasekharcse522/Movie-Review-NLP-Pipeline"


Repository chandrasekharcse522/Movie-Review-NLP-Pipeline initialized!

2025-09-24 13:26:06,604 - INFO - Repository chandrasekharcse522/Movie-Review-NLP-Pipeline initialized!
2025-09-24 13:26:25,848 - WARNING - Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /chandrasekharcse522/Movie-Review-NLP-Pipeline.mlflow/api/2.0/mlflow/experiments/get-by-name?experiment_name=Movie-Review+Logistic+Regression+Baseline


<Experiment: artifact_location='mlflow-artifacts:/88104d314b2f4c658a48a58c797bfb21', creation_time=1758699471296, experiment_id='0', last_update_time=1758699471296, lifecycle_stage='active', name='Movie-Review Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.perf_counter()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.perf_counter()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-09-24 13:26:58,405 - INFO - Starting MLflow run...
2025-09-24 13:26:58,884 - INFO - Logging preprocessing parameters...
2025-09-24 13:27:00,219 - INFO - Initializing Logistic Regression model...
2025-09-24 13:27:00,222 - INFO - Fitting the model...
2025-09-24 13:27:00,270 - INFO - Model training complete.
2025-09-24 13:27:00,271 - INFO - Logging model parameters...
2025-09-24 13:27:00,689 - INFO - Making predictions...
2025-09-24 13:27:00,698 - INFO - Calculating evaluation metrics...
2025-09-24 13:27:00,757 - INFO - Logging evaluation metrics...
2025-09-24 13:27:02,436 - INFO - Saving and logging the model...
c:\Users\Chandra\miniconda3\envs\nlp-movie-review\lib\site-packages\_distutils_hack\__init__.py:15: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed i